In [1]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
import math
from numpy import lexsort

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches

from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

# Load data data
ground_truth_poses = np.loadtxt("../data/experiments/pose_scanner_leica_affine.txt") # id,x,y,z,w,x,y,z
ground_truth_poses = np.array([ground_truth_poses]).T        

corrected_poses = np.loadtxt("../data/experiments/corrected_poses_360fov_1200_run2.txt")              # idA,idB,x,y,z,w,x,y,z
corrected_poses = np.array([corrected_poses]).T

results = np.loadtxt("../data/experiments/compare_results_360fov_1200_run2.txt")                     # idA,idB,
results = np.array([results]).T                                                 # fov_overlap, octree_overlap
                                                                                    # alignability, alignment_risk
                                                                                    # degeneracy, ICN
# ground_truth_poses
# print results

In [2]:
# Compute translation error
def translationError(xg, yg, zg, xc, yc, zc):
   
    trans_err = np.zeros((xc.shape[0],1))
    gt_index = 0
    for i in range(0, xc.shape[0]):
        if (gt_index > xg.shape[0]-1):
            gt_index = 0
        
#         print gt_index, i
  
        trans_err[i] =  math.sqrt( pow((xg[gt_index] - xc[i]),2.0) + 
                                   pow((yg[gt_index] - yc[i]),2.0) +
                                   pow((zg[gt_index] - zc[i]),2.0) );
        gt_index = gt_index+1
    return trans_err
49
transl_err = translationError(ground_truth_poses[1], # [col][row]
                              ground_truth_poses[2],
                              ground_truth_poses[3],
                              corrected_poses[2],
                              corrected_poses[3],
                              corrected_poses[4])

# transl_err

In [3]:
# Create Labels given ground truth and errors
transl_thresh = 0.02 # [m]
rot_thresh = 30

def createLabels(transl_err):
    
    labels = np.zeros((transl_err.shape[0],1))

    for i in range(0, labels.shape[0]):
        if transl_err[i] > transl_thresh: #and rot_err[i] > rot_thresh:
            labels[i] = 1.0

    return labels

labels = createLabels(transl_err)

# print labels.shape

In [4]:
probs_ours = results[5]
probs_degeneracy = results[6]
probs_icn = results[7]

risk_threshold = 0.50

overlap = results[3]
shape = overlap.shape
num_bins = 10
# Accuracy vs. Overlap
bin_acc_ours = np.zeros((num_bins,1))
for i in range(1, 11):
    bin_max = num_bins*i
    count = 0
    true = 0
#     print 'bin_max =', bin_max
    for j in range(0, shape[0]):
#         print 'j =', j
#         print 'overlap[j] =', overlap[j], overlap[j] <= bin_max
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            count = count + 1
#             print 'labels[j] =', labels[j], probs_ours[j]>risk_threshold, probs_ours[j]<=risk_threshold
            if (labels[j]==1.0 and probs_ours[j]>risk_threshold) or (labels[j]==0.0 and probs_ours[j]<=risk_threshold):
#             if (labels[j]==1.0 and probs_ours[j]<risk_threshold):
                # true positive or true negative
                true = true + 1
    bin_acc_ours[i-1] = 100.0*(float(true) / float(count))
    
# print bin_acc_ours

In [5]:
# Precision-Recall vs. Overlap
bin_recall_ours = np.zeros((num_bins,1))
bin_precision_ours = np.zeros((num_bins,1))
bin_fscore_ours = np.zeros((num_bins,1))
# Recall
for i in range(1, 11):
    bin_max = num_bins*i
    cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (labels[j]==1.0):
                # condition positive
                cond_positive = cond_positive + 1
                if (probs_ours[j]>risk_threshold):
                    # true positive
                    true = true + 1
#     print float(true)
#     print float(cond_positive)
    if cond_positive>0:
        bin_recall_ours[i-1] = (float(true) / float(cond_positive))
    
# print bin_recall_ours

# Precision
for i in range(1, 11):
    bin_max = num_bins*i
    pred_cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (probs_ours[j]>risk_threshold):
                # predicted condition positive
                pred_cond_positive = pred_cond_positive + 1
                if (labels[j]==1.0):
                    # true positive
                    true = true + 1
#     print float(pred_cond_positive)
#     print float(true)
    if true>0:
        bin_precision_ours[i-1] = (float(true) / float(pred_cond_positive))
    
# print bin_precision_ours

# F-Score
for i in range(0, 10):
    if bin_recall_ours[i]>0:
        den1 = (1.0 / bin_recall_ours[i])
    else:
        den1 = 1.0
    if bin_precision_ours[i]>0:
        den2 = (1.0 / bin_precision_ours[i])
    else:
        den2 = 1.0
    bin_fscore_ours[i] = 2.0 / ( den1 + den2 )
    
print bin_fscore_ours

[[ 1.        ]
 [ 0.9071618 ]
 [ 0.72727273]
 [ 0.4057971 ]
 [ 0.34782609]
 [ 0.2       ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]


In [6]:
deg_threshold = 0.05

overlap = results[3]
shape = overlap.shape
num_bins = 10
bin_acc_deg = np.zeros((num_bins,1))
for i in range(1, 11):
    bin_max = num_bins*i
    count = 0
    true = 0
#     print 'bin_max =', bin_max
    for j in range(0, shape[0]):
#         print 'j =', j
#         print 'overlap[j] =', overlap[j], overlap[j] <= bin_max
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            count = count + 1
#             print 'labels[j] =', labels[j], probs_degeneracy[j]>risk_threshold, probs_degeneracy[j]<=risk_threshold
            if (labels[j]==1.0 and probs_degeneracy[j]<deg_threshold) or (labels[j]==0.0 and probs_degeneracy[j]>=deg_threshold):
#             if (labels[j]==1.0 and probs_degeneracy[j]>deg_threshold):
                # true positive or true negative
                true = true + 1
    bin_acc_deg[i-1] = 100.0*(float(true) / float(count))
    
# print bin_acc_deg

In [7]:
# Precision-Recall vs. Overlap
bin_recall_deg = np.zeros((num_bins,1))
bin_precision_deg = np.zeros((num_bins,1))
bin_fscore_deg = np.zeros((num_bins,1))
# Recall
for i in range(1, 11):
    bin_max = num_bins*i
    cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (labels[j]==1.0):
                # condition positive
                cond_positive = cond_positive + 1
                if (probs_degeneracy[j]<deg_threshold):
                    # true positive
                    true = true + 1
#     print float(true)
#     print float(cond_positive)
    if cond_positive>0:
        bin_recall_deg[i-1] = (float(true) / float(cond_positive))
    
# print bin_recall_deg

# Precision
for i in range(1, 11):
    bin_max = num_bins*i
    pred_cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (probs_degeneracy[j]<deg_threshold):
                # predicted condition positive
                pred_cond_positive = pred_cond_positive + 1
                if (labels[j]==1.0):
                    # true positive
                    true = true + 1
#     print float(pred_cond_positive)
#     print float(true)
    if true>0:
        bin_precision_deg[i-1] = (float(true) / float(pred_cond_positive))
    
# print bin_precision_deg

# F-Score
for i in range(0, 10):
    if bin_recall_deg[i]>0:
        den1 = (1.0 / bin_recall_deg[i])
    else:
        den1 = 1.0
    if bin_precision_deg[i]>0:
        den2 = (1.0 / bin_precision_deg[i])
    else:
        den2 = 1.0
    bin_fscore_deg[i] = 2.0 / ( den1 + den2 )
    
print bin_fscore_deg

[[ 0.87651332]
 [ 0.66013072]
 [ 0.45751634]
 [ 0.53846154]
 [ 0.29411765]
 [ 0.26086957]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]


In [8]:
icn_threshold = 0.05

overlap = results[3]
shape = overlap.shape
num_bins = 10
bin_acc_icn = np.zeros((num_bins,1))
for i in range(1, 11):
    bin_max = num_bins*i
    count = 0
    true = 0
#     print 'bin_max =', bin_max
    for j in range(0, shape[0]):
#         print 'j =', j
#         print 'overlap[j] =', overlap[j], overlap[j] <= bin_max
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            count = count + 1
#             print 'labels[j] =', labels[j], probs_icn[j]>risk_threshold, probs_icn[j]<=risk_threshold
            if (labels[j]==1.0 and probs_icn[j]<icn_threshold) or (labels[j]==0.0 and probs_icn[j]>=icn_threshold):
#             if (labels[j]==1.0 and probs_icn[j]>icn_threshold):
                # true positive or true negative
                true = true + 1
    bin_acc_icn[i-1] = 100.0*(float(true) / float(count))
    
# print bin_acc_icn

In [9]:
# Precision-Recall vs. Overlap
bin_recall_icn = np.zeros((num_bins,1))
bin_precision_icn = np.zeros((num_bins,1))
bin_fscore_icn = np.zeros((num_bins,1))
# Recall
for i in range(1, 11):
    bin_max = num_bins*i
    cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (labels[j]==1.0):
                # condition positive
                cond_positive = cond_positive + 1
                if (probs_icn[j]<icn_threshold):
                    # true positive
                    true = true + 1
#     print float(true)
#     print float(cond_positive)
    if cond_positive>0:
        bin_recall_icn[i-1] = (float(true) / float(cond_positive))
    
# print bin_recall_icn

# Precision
for i in range(1, 11):
    bin_max = num_bins*i
    pred_cond_positive = 0
    true = 0
    for j in range(0, shape[0]):
        if (overlap[j] <= bin_max and overlap[j] > (bin_max-num_bins)):
            if (probs_icn[j]<icn_threshold):
                # predicted condition positive
                pred_cond_positive = pred_cond_positive + 1
                if (labels[j]==1.0):
                    # true positive
                    true = true + 1
#     print float(pred_cond_positive)
#     print float(true)
    if true>0:
        bin_precision_icn[i-1] = (float(true) / float(pred_cond_positive))
    
# print bin_precision_icn

# F-Score
for i in range(0, 10):
    if bin_recall_icn[i]>0:
        den1 = (1.0 / bin_recall_icn[i])
    else:
        den1 = 1.0
    if bin_precision_icn[i]>0:
        den2 = (1.0 / bin_precision_icn[i])
    else:
        den2 = 1.0
    bin_fscore_icn[i] = 2.0 / ( den1 + den2 )
    
print bin_fscore_icn

[[ 0.83709273]
 [ 0.72049689]
 [ 0.55813953]
 [ 0.52525253]
 [ 0.37037037]
 [ 0.15      ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]


In [41]:
from scipy.interpolate import spline
from scipy.ndimage.interpolation import shift


# plot accuracy when overlap [0:10],[10,20],...
bins = [0,10,20,30,40,50,60,70,80,90,100,110]

xnew = np.linspace(0,105,105) #300 represents number of points to make between T.min and T.max


first_ours = np.expand_dims(bin_acc_ours[0], axis=0)
last_ours = np.expand_dims(bin_acc_ours[9], axis=0)
conc_ours = np.concatenate((first_ours, bin_acc_ours), axis=0)
conc_ours = np.concatenate((conc_ours, last_ours), axis=0)

first_deg = np.expand_dims(bin_acc_deg[0], axis=0)
last_deg = np.expand_dims(bin_acc_deg[9], axis=0)
conc_deg = np.concatenate((first_deg, bin_acc_deg), axis=0)
conc_deg = np.concatenate((conc_deg, last_deg), axis=0)

first_icn = np.expand_dims(bin_acc_icn[0], axis=0)
last_icn = np.expand_dims(bin_acc_icn[9], axis=0)
conc_icn = np.concatenate((first_icn, bin_acc_icn), axis=0)
conc_icn = np.concatenate((conc_icn, last_icn), axis=0)

conc_ours_smooth = spline(bins,conc_ours,shift(xnew,-5))
conc_deg_smooth = spline(bins,conc_deg,shift(xnew,-5))
conc_icn_smooth = spline(bins,conc_icn,shift(xnew,-5))

plt.plot(xnew,conc_ours_smooth,label = 'AR',linewidth=2.0)
plt.plot(xnew,conc_deg_smooth,label = 'D',linewidth=2.0)
plt.plot(xnew,conc_icn_smooth,label = 'ICN',linewidth=2.0)



# our_lines = plt.step(bins,conc_ours,'b',linewidth=2.0,label = 'AR', alpha=1)
# # plt.fill_between(our_lines[0].get_xdata(orig=False), 0, our_lines[0].get_ydata(orig=False), color = 'red', alpha=1)
# deg_lines = plt.step(bins,conc_deg,'g',linewidth=2.0,label = 'Deg', alpha=1)
# # # plt.fill_between(deg_lines[0].get_xdata(orig=False), 0, deg_lines[0].get_ydata(orig=False), color = 'blue', alpha=0.5)
# icn_lines = plt.step(bins,conc_icn,'r',linewidth=2.0,label = 'ICN', alpha=1)
# # # plt.fill_between(icn_lines[0].get_xdata(orig=False), 0, icn_lines[0].get_ydata(orig=False), color = 'green', alpha=0.5)
plt.xlabel('Overlap [%]')
plt.ylabel('Accuracy [%]')
plt.xticks(np.arange(0, 101, 10))
plt.yticks(np.arange(0, 101, 10))
plt.ylim(30,100)
plt.xlim(0,100)
plt.legend(loc = 'lower right')
# plt.text(8, 90, 'AR', color='black', horizontalalignment='center', fontsize=15)
# plt.text(8, 31, 'Deg', color='black', horizontalalignment='center', fontsize=15)
# plt.text(8, 50, 'ICN', color='black', horizontalalignment='center', fontsize=15)
plt.grid()
plt.show()

In [11]:
# plot fscore when overlap [0:10],[10,20],...
bins = [0,10,20,30,40,50,60,70,80,90,100]

first_ours = np.expand_dims(bin_fscore_ours[0], axis=0)
conc_ours = np.concatenate((first_ours, bin_fscore_ours), axis=0)

first_deg = np.expand_dims(bin_fscore_deg[0], axis=0)
conc_deg = np.concatenate((first_deg, bin_fscore_deg), axis=0)

first_icn = np.expand_dims(bin_fscore_icn[0], axis=0)
conc_icn = np.concatenate((first_icn, bin_fscore_icn), axis=0)

our_lines = plt.step(bins,conc_ours,'r',linewidth=2.0,label = 'AR', alpha=1)
# plt.fill_between(our_lines[0].get_xdata(orig=False), 0, our_lines[0].get_ydata(orig=False), color = 'red', alpha=0)
deg_lines = plt.step(bins,conc_deg,'b',linewidth=2.0,label = 'Deg', alpha=1)
# plt.fill_between(deg_lines[0].get_xdata(orig=False), 0, deg_lines[0].get_ydata(orig=False), color = 'blue', alpha=0)
icn_lines = plt.step(bins,conc_icn,'g',linewidth=2.0,label = 'ICN', alpha=1)
# plt.fill_between(icn_lines[0].get_xdata(orig=False), 0, icn_lines[0].get_ydata(orig=False), color = 'green', alpha=0)
plt.xlabel('Overlap [%]')
plt.ylabel('F1-Score [%]')
plt.xticks(np.arange(0, 101, 10))
plt.yticks(np.arange(0, 1.1, 10))
plt.ylim(0,1.1)
# plt.legend(loc = 'lower right')
plt.text(8, 90, 'AR', color='black', horizontalalignment='center', fontsize=15)
plt.text(8, 31, 'Deg', color='black', horizontalalignment='center', fontsize=15)
plt.text(8, 50, 'ICN', color='black', horizontalalignment='center', fontsize=15)
plt.grid()
plt.show()